<a href="https://colab.research.google.com/github/Kobayashi139/food-supply/blob/main/%E3%83%91%E3%83%AC%E3%83%BC%E3%83%88%E3%83%95%E3%83%AD%E3%83%B3%E3%83%86%E3%82%A3%E3%82%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ライブラリのインポート

In [1]:
# =================================================================
# モジュール1: 環境設定とライブラリのインポート
# =================================================================
!pip install -q pyomo plotly
!apt-get install -y -qq glpk-utils

import pyomo.environ as pyo
import pandas as pd
import numpy as np
import random, time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

try:
    # 乱数シードを固定し、毎回同じ乱数が発生するようにする
    np.random.seed(42)
    solver = pyo.SolverFactory('glpk')
except pyo.common.errors.ApplicationError:
    print("GLPK solver not found. Please ensure it is installed and in your system's PATH.")
    solver = None


Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

### パラメータとシナリオ

In [2]:
# =================================================================
# モジュール2: パラメータとシナリオの定義
# =================================================================

# --- 製品タイプの集合 ---
# I = {f: 生鮮品（檸檬）, a: 加工品A（檸檬タルト）, b: 加工品B（檸檬シロップ）}
products = ['f', 'a', 'b']
processed_products = ['a', 'b']

# --- 単位統一のための基本データ設定 ---
# 製品1個あたりの基本情報
# タルト('a')
WEIGHT_TART_g = 350      # 容量 (g/個)
LEMON_PER_TART_ml = 40    # 檸檬含量 (ml/個)
LIFESPAN_TART_days = 120   # 保存期間 (日)
SALES_TART = 3800 # 値段 (円/個)
# シロップ('b')
WEIGHT_SYRUP_g = 400     # 容量 (g/個)
LEMON_PER_SYRUP_ml = 200  # 檸檬含量 (ml/個)
LIFESPAN_SYRUP_days = 365 # 保存期間 (日)
SALES_SYRUP = 2200 # 値段 (円/個)
# 生鮮品('f') - 檸檬
LIFESPAN_FRESH_days = 60  # 保存期間 (日、仮定)
SALES_FRESH = 500 # 値段 (円/kg)

# 1個あたりの重量 (kg/個)
WEIGHT_PER_PIECE = {
    'a': WEIGHT_TART_g / 1000,
    'b': WEIGHT_SYRUP_g / 1000
}
# 製品1個の製造に必要な檸檬の重量 (kg/個) - 檸檬の密度を約1g/mlと仮定
LEMON_KG_PER_PIECE = {
    'a': LEMON_PER_TART_ml / 1000,
    'b': LEMON_PER_SYRUP_ml / 1000
}

# 電気代・ガス代 (円/月)
COST_FREEZER_monthly = 389.17 # 冷凍庫
COST_OVEN_monthly = 155       # オーブン
COST_GAS_monthly = 1048.5     # ガスコンロ

# 梱包費など (円/個)
PACKING_COST_PER_PIECE = {
    'a': 63,
    'b': 230
}

# --- コスト、劣化率、効率などのパラメータ ---

# 1. 保管コスト h (円/kg・月)
# 月額コストを、その保管場所で保管可能な最大重量(kg)で割って算出
h_a = COST_FREEZER_monthly / 80  # 冷凍庫100Lに80kg保管可能と仮定
h_b = 1                          # 常温保管コストは微小と仮定
h_f = 5                          # 冷蔵庫の月額コスト

# 2. 加工コスト p (円/kg)
# 製品1kgを生産するためのコスト（設備費＋梱包費）
# タルト('a'): 月産60個と仮定
p_a_monthly_cost = COST_OVEN_monthly + (PACKING_COST_PER_PIECE['a'] * 60)
p_a_monthly_weight_kg = WEIGHT_PER_PIECE['a'] * 60
p_a = p_a_monthly_cost / p_a_monthly_weight_kg
# シロップ('b'): 月産30個と仮定
p_b_monthly_cost = COST_GAS_monthly + (PACKING_COST_PER_PIECE['b'] * 30)
p_b_monthly_weight_kg = WEIGHT_PER_PIECE['b'] * 30
p_b = p_b_monthly_cost / p_b_monthly_weight_kg

# 3. 劣化率 gamma (%/月)
# 1ヶ月(30日)で劣化する割合。 (30日 / 保存期間) で計算、上限1.0(100%)
gamma_f = min(1.0, 30 / LIFESPAN_FRESH_days)
gamma_a = min(1.0, 30 / LIFESPAN_TART_days)
gamma_b = min(1.0, 30 / LIFESPAN_SYRUP_days)

# h_i: 保管コスト, p_i: 加工コスト, γ_i: 劣化率, φ_i: 廃棄コスト, e_i: 加工効率, o: 1単位あたりの機会損失コスト
params = {
    'h': {'f': h_f, 'a': h_a, 'b': h_b},
    'p': {'f': 0, 'a': p_a, 'b': p_b},
    'gamma': {'f': gamma_f, 'a': gamma_a, 'b': gamma_b},
    'e': {'f': 1, 'a': 0.8 ,'b': 0.9},
    'phi': {'f': 1000, 'a': 1000, 'b': 1500}, # 生鮮品の処理コスト重視(円/kg)
    'phi_pre': {'a': 1000, 'b': 1000},
    'o': {'f': 500, 'a': 2000, 'b': 1500}  # 粗利寄りの計上(円/kg)
}

# --- 能力に関するパラメータ (単位: kg) ---
capacities = {
    # U: 在庫上限 (kg)
    'U': {'f': 40, 'a': 80, 'b': 120},
    # u: 月間の最大生産量 (kg/月)
    'u': {
        'a': 60 * WEIGHT_PER_PIECE['a'], # 月最大60個生産と仮定
        'b': 30 * WEIGHT_PER_PIECE['b']   # 月最大30個生産と仮定
    }
}

# --- 供給と初期在庫 (単位: kg) ---
C_supply = 25 # 月間の生鮮品(檸檬)の供給量 (kg)
initial_inventory_post = {'f': 5, 'a': 0, 'b': 0} # 加工後在庫 (kg)
initial_inventory_pre = {'a': 0, 'b': 0}          # 加工前在庫(原料檸檬) (kg)

# --- 需要パラメータ ---
demand_params = {
    'mean': {
        'f': 5, # 生の檸檬の月間需要 (kg)
        'a': 30 * WEIGHT_PER_PIECE['a'], # タルト: 月需要(個
        'b': 20 * WEIGHT_PER_PIECE['b']  # シロップ: 月需要(個
    },
    'std': {
        'f': 2,
        'a': 15 * WEIGHT_PER_PIECE['a'],
        'b': 10 * WEIGHT_PER_PIECE['b']
    }
}

num_scenarios_per_period = 20 # 生成するシナリオの数

params['waste_limit'] = 0.5 * C_supply

# --- 環境負荷係数 (単位: ポイント/kg) ---
# 廃棄物1kgあたり、または加工プロセスで原料1kg使用あたりの負荷
waste_emission_factors = {
    'low': {'f': 1.0, 'a': 1.5, 'b': 2.0},
    'middle':{'f': 4.0, 'a': 4.5, 'b': 5.0},
    'high': {'f': 8.0, 'a': 8.5, 'b': 9.0}
}
process_emission_factors = {
    'low': {'a': 1.8, 'b': 2.5},
    'middle':{'a': 4.5, 'b': 3.0},
    'high': {'a': 6.0, 'b': 4.5}
}


## パレートフロンティア

In [3]:
# =================================================================
# モジュール3: 多目的最適化モデルを解く関数 (改善版)
# =================================================================
def solve_multi_objective_model(initial_inv_post, initial_inv_pre, supply, scenarios, params, caps, env_weight):
    model = pyo.ConcreteModel("Multi_Objective_Model")

    # 集合
    model.I = pyo.Set(initialize=products)
    model.I_processed = pyo.Set(initialize=processed_products)
    model.S = pyo.Set(initialize=scenarios.keys())

    # 変数
    model.X = pyo.Var(model.I, domain=pyo.NonNegativeReals) # 原料配分量
    model.P_actual = pyo.Var(model.I_processed, domain=pyo.NonNegativeReals) # 加工投入量
    model.I_end_post = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 期末在庫(加工後)
    model.I_end_pre = pyo.Var(model.S, model.I_processed, domain=pyo.NonNegativeReals) # 期末在庫(加工前)
    model.S_sold = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 販売量
    model.W_waste_post = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 廃棄量(加工後)
    model.W_waste_pre = pyo.Var(model.S, model.I_processed, domain=pyo.NonNegativeReals) # 廃棄量(加工前)
    model.UnmetDemand = pyo.Var(model.S, model.I, domain=pyo.NonNegativeReals) # 未充足需要

    # --- 目的関数（経済コスト＋環境負荷×重み） ---
    economic_cost_expr = (
        # 保管コスト
        sum(scenarios[s]['prob'] * sum(params['h'][i] * model.I_end_post[s, i] for i in model.I) for s in model.S)
        # 加工コスト
        + sum(params['p'][i] * model.P_actual[i] for i in model.I_processed)
        # 廃棄コスト (加工後)
        + sum(scenarios[s]['prob'] * sum(params['phi'][i] * model.W_waste_post[s, i] for i in model.I) for s in model.S)
        # ★改善点: 廃棄コスト (加工前) を追加
        + sum(scenarios[s]['prob'] * sum(params['phi_pre'][i] * model.W_waste_pre[s, i] for i in model.I_processed) for s in model.S)
        # 機会損失コスト
        + sum(scenarios[s]['prob'] * sum(params['o'][i] * model.UnmetDemand[s, i] for i in model.I) for s in model.S)
    )
    environmental_score_expr = (
        # 廃棄による環境負荷 (加工後)
        sum(scenarios[s]['prob'] * sum(waste_emission_factors['middle'][p] * model.W_waste_post[s, p] for p in model.I) for s in model.S)
        # ★改善点: 廃棄による環境負荷 (加工前) を追加 (生鮮品の係数'f'を使用)
        + sum(scenarios[s]['prob'] * sum(waste_emission_factors['middle']['f'] * model.W_waste_pre[s, i] for i in model.I_processed) for s in model.S)
        # 加工による環境負荷
        + sum(process_emission_factors['middle'][p] * model.P_actual[p] for p in model.I_processed)
    )

    # 経済コスト
    model.economic_cost = pyo.Expression(expr=economic_cost_expr)
    # 環境負荷スコア
    model.environmental_score = pyo.Expression(expr=environmental_score_expr)
    # 目的関数 = 経済的コスト + λ × 環境負荷スコア
    model.total_objective = pyo.Objective(
        expr=model.economic_cost + env_weight * model.environmental_score,
        sense=pyo.minimize
    )
    # model.total_objectiveを数値として表示できるようにする、目的関数自体で比較しないといけない

    # 制約
    model.constraints = pyo.ConstraintList()

    # 総供給量制約
    model.constraints.add(sum(model.X[i] for i in model.I) == supply)

    # 加工能力と原料利用の制約
    for i in model.I_processed:
        # 月間最大生産量
        model.constraints.add(model.P_actual[i] * params['e'][i] <= caps['u'][i])
        # 加工投入量は、(繰越在庫＋新規割当)を超えられない
        model.constraints.add(model.P_actual[i] <= initial_inv_pre.get(i, 0) * (1 - params['gamma']['f']) + model.X[i])

    # 各シナリオにおけるバランス制約
    for s in model.S:
        # 在庫バランス (加工後) - 生鮮品
        model.constraints.add(initial_inv_post['f'] * (1 - params['gamma']['f']) + model.X['f'] == model.S_sold[s, 'f'] + model.I_end_post[s, 'f'])

        for i in model.I_processed:
            # 在庫バランス (加工後) - 加工品
            model.constraints.add(initial_inv_post.get(i, 0) * (1 - params['gamma'][i]) + model.P_actual[i] * params['e'][i] == model.S_sold[s, i] + model.I_end_post[s, i])

            # ★エラー修正 & 改善点: 在庫バランス (加工前)
            # (期首在庫(劣化考慮) + 当期割当) - 加工投入量 = 期末在庫
            model.constraints.add(
                (initial_inv_pre.get(i, 0) * (1 - params['gamma']['f']) + model.X[i]) - model.P_actual[i] == model.I_end_pre[s, i]
            )
            # ★改善点: 廃棄量計算 (加工前)
            model.constraints.add(model.W_waste_pre[s, i] == initial_inv_pre.get(i, 0) * params['gamma']['f'])

        for i in model.I:
            # 販売量は需要以下
            model.constraints.add(model.S_sold[s, i] <= scenarios[s]['d'][i])
            # 未充足需要の計算
            model.constraints.add(model.S_sold[s, i] + model.UnmetDemand[s, i] == scenarios[s]['d'][i])
            # 廃棄量計算 (加工後)
            model.constraints.add(model.W_waste_post[s, i] == initial_inv_post.get(i, 0) * params['gamma'][i])
            # 在庫上限
            model.constraints.add(model.I_end_post[s, i] <= caps['U'][i])

    if solver is None:
        return None, None

    results = solver.solve(model, tee=False)
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        decisions = {
            'X': {i: pyo.value(model.X[i]) for i in model.I},
            'P_actual': {i: pyo.value(model.P_actual[i]) for i in model.I_processed}
        }
        return decisions, model
    else:
        return None, None

# =================================================================
# モジュール4: ローリングホライズン・シミュレーション本体
# =================================================================
def run_rolling_horizon_simulation(simulation_periods, env_weight, initial_inventory_post, initial_inventory_pre, master_scenarios, verbose=True):
    if verbose:
        print("\n" + "="*80)
        print(f"詳細シミュレーションを開始します (環境重視度 λ = {env_weight:.2f})")
        print("="*80)

    inv_post = initial_inventory_post.copy()
    inv_pre = initial_inventory_pre.copy()

    full_history = []
    for t in range(1, simulation_periods + 1):
        # シナリオの生成
        scenarios = master_scenarios[t-1]

        decisions, solved_model = solve_multi_objective_model(inv_post, inv_pre, C_supply, scenarios, params, capacities, env_weight)

        if not decisions:
            print(f"❌ 期 {t} で最適解なし。中断。")
            break

        # 結果を記録
        period_data = {
            'period': t,
            'cost': pyo.value(solved_model.economic_cost),
            'env_score': pyo.value(solved_model.environmental_score),
            'weight': env_weight,
            # 意思決定変数
            'X_f': decisions['X']['f'], 'X_a': decisions['X']['a'], 'X_b': decisions['X']['b'],
            'P_a': decisions['P_actual']['a'], 'P_b': decisions['P_actual']['b'],
            # 在庫量 (期待値)
            'Inv_post_f': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[s, 'f']) for s in scenarios),
            'Inv_post_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[s, 'a']) for s in scenarios),
            'Inv_post_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[s, 'b']) for s in scenarios),
            'Inv_pre_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_pre[s, 'a']) for s in scenarios),
            'Inv_pre_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_pre[s, 'b']) for s in scenarios),
            # 廃棄量 (期待値)
            'W_post_f': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[s, 'f']) for s in scenarios),
            'W_post_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[s, 'a']) for s in scenarios),
            'W_post_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[s, 'b']) for s in scenarios),
            'W_pre_a': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_pre[s, 'a']) for s in scenarios),
            'W_pre_b': sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_pre[s, 'b']) for s in scenarios),
        }
        full_history.append(period_data)

        # 次の期の初期在庫を更新
        inv_post = {p: period_data[f'Inv_post_{p}'] for p in products}
        inv_pre = {p: period_data[f'Inv_pre_{p}'] for p in processed_products}

    return pd.DataFrame(full_history)

# =================================================================
# モジュール5: パレートフロンティア分析
# =================================================================
def run_pareto_analysis(env_weight_list, simulation_periods, master_scenarios, title="パレートフロンティア分析"):
    print("="*80 + f"\n{title}を開始...\n" + "="*80)
    results_list = []
    start_time = time.time()

    for weight in env_weight_list:
        # print(f"分析中... 環境重視度 (λ) = {weight:.2f}")
        df_history = run_rolling_horizon_simulation(simulation_periods, env_weight=weight,
                                                  initial_inventory_post=initial_inventory_post,
                                                  initial_inventory_pre=initial_inventory_pre,
                                                  master_scenarios=master_scenarios,
                                                  verbose=False)

        if not df_history.empty:
            total_cost = df_history['cost'].sum()
            total_env_score = df_history['env_score'].sum()
            total_objective = total_cost + weight * total_env_score
            results_list.append({
                'weight': weight, 'total_cost': total_cost,
                'total_env_score': total_env_score, 'total_objective': total_objective
            })

    print(f"\n完了: {time.time() - start_time:.2f} 秒")
    return pd.DataFrame(results_list)

# =================================================================
# モジュール6: 1期間の詳細分析と結果表示
# =================================================================
def analyze_and_print_period_details(model, scenarios, params):
    holding_cost = sum(scenarios[s]['prob'] * sum(params['h'][i] * pyo.value(model.I_end_post[s, i]) for i in model.I) for s in model.S)
    processing_cost = sum(params['p'][i] * pyo.value(model.P_actual[i]) for i in model.I_processed)
    waste_cost_post = sum(scenarios[s]['prob'] * sum(params['phi'][i] * pyo.value(model.W_waste_post[s, i]) for i in model.I) for s in model.S)
    waste_cost_pre = sum(scenarios[s]['prob'] * sum(params['phi_pre'][i] * pyo.value(model.W_waste_pre[s, i]) for i in model.I_processed) for s in model.S)
    total_waste_cost = waste_cost_post + waste_cost_pre
    opportunity_loss_cost = sum(scenarios[s]['prob'] * sum(params['o'][i] * pyo.value(model.UnmetDemand[s, i]) for i in model.I) for s in model.S)
    total_cost = pyo.value(model.economic_cost)
    spoilage_f = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[s, 'f']) for s in model.S)
    spoilage_post_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[s, 'a']) for s in model.S)
    spoilage_pre_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[s, 'a']) for s in model.S)
    processing_loss_a = pyo.value(model.P_actual['a']) * (1 - params['e']['a'])
    spoilage_post_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[s, 'b']) for s in model.S)
    spoilage_pre_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[s, 'b']) for s in model.S)
    processing_loss_b = pyo.value(model.P_actual['b']) * (1 - params['e']['b'])
    total_spoilage = spoilage_f + spoilage_post_a + spoilage_pre_a + spoilage_post_b + spoilage_pre_b
    total_processing_loss = processing_loss_a + processing_loss_b
    total_food_loss = total_spoilage + total_processing_loss
    env_scores = {}
    for level in ['low', 'middle', 'high']:
        waste_impact = sum(scenarios[s]['prob'] * (
            sum(waste_emission_factors[level][i] * pyo.value(model.W_waste_post[s, i]) for i in model.I) +
            sum(waste_emission_factors[level]['f'] * pyo.value(model.W_waste_pre[s, i]) for i in model.I_processed)
        ) for s in model.S)
        process_impact = sum(process_emission_factors[level][i] * pyo.value(model.P_actual[i]) for i in model.I_processed)
        env_scores[level] = {'total': waste_impact + process_impact, 'waste': waste_impact, 'process': process_impact}
    print(f"期待総コスト: {total_cost:,.3f} 円\n")
    print("--- 総コストの内訳 ---")
    print(f"  期待保管コスト        : {holding_cost:,.3f}")
    print(f"  加工コスト            : {processing_cost:,.3f}")
    print(f"  期待廃棄コスト        : {total_waste_cost:,.3f} (加工後 {waste_cost_post:,.3f}, 加工前 {waste_cost_pre:,.3f})")
    print(f"  期待機会損失コスト    : {opportunity_loss_cost:,.3f}")
    print("-----------------------------")
    print(f"  内訳の合計            : {(holding_cost + processing_cost + total_waste_cost + opportunity_loss_cost):,.3f}\n")
    print("--- 食品ロス（廃棄量）の内訳 ---")
    print("【生鮮品】")
    print(f"  - 劣化ロス (f)   : {spoilage_f:.3f} kg\n")
    print("【加工品】")
    print("  - 製品 A:")
    print(f"    - 劣化ロス   : {(spoilage_post_a + spoilage_pre_a):.3f} kg (加工後 {spoilage_post_a:.3f}, 加工前 {spoilage_pre_a:.3f})")
    print(f"    - 加工ロス   : {processing_loss_a:.3f} kg")
    print("  - 製品 B:")
    print(f"    - 劣化ロス   : {(spoilage_post_b + spoilage_pre_b):.3f} kg (加工後 {spoilage_post_b:.3f}, 加工前 {spoilage_pre_b:.3f})")
    print(f"    - 加工ロス   : {processing_loss_b:.3f} kg")
    print("-----------------------------")
    print(f"総食品ロス量（劣化+加工）: {total_food_loss:.3f} kg")
    print(f"    内訳 - 劣化ロス : {total_spoilage:.3f} kg")
    print(f"         - 加工ロス : {total_processing_loss:.3f} kg\n")
    print("--- 環境負荷の相対的評価 ---")
    for level, scores in env_scores.items():
        print(f"  {level.capitalize():<7} 負荷: {scores['total']:.3f} ポイント (廃棄 {scores['waste']:.3f}, 加工 {scores['process']:.3f})")

# =================================================================
# モジュール7: 詳細シミュレーション結果の可視化ダッシュボード
# =================================================================
def create_dashboard(df):
    if df.empty:
        print("データが空のため、ダッシュボードを作成できません。")
        return
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("在庫レベルの推移 (kg)", "期間ごとの生産量 (kg)",
                        "期間ごとの廃棄量 (kg)", "原料配分率 (%)"),
        specs=[[{}, {}], [{}, {'type': 'domain'}]]
    )
    inv_post_cols, inv_pre_cols = ['Inv_post_f', 'Inv_post_a', 'Inv_post_b'], ['Inv_pre_a', 'Inv_pre_b']
    colors = px.colors.qualitative.Plotly
    for i, col in enumerate(inv_post_cols):
        fig.add_trace(go.Bar(x=df['period'], y=df[col], name=f'加工後-{col[-1]}', legendgroup='inv_post', marker_color=colors[i]), row=1, col=1)
    for i, col in enumerate(inv_pre_cols):
         fig.add_trace(go.Bar(x=df['period'], y=df[col], name=f'加工前-{col[-1]}', legendgroup='inv_pre', marker_color=colors[i+3], opacity=0.7), row=1, col=1)
    fig.update_layout(barmode='stack', bargap=0.1)
    fig.add_trace(go.Bar(x=df['period'], y=df['P_a'], name='タルト(a)生産', marker_color=colors[1]), row=1, col=2)
    fig.add_trace(go.Bar(x=df['period'], y=df['P_b'], name='シロップ(b)生産', marker_color=colors[2]), row=1, col=2)
    waste_cols = ['W_post_f', 'W_post_a', 'W_post_b', 'W_pre_a', 'W_pre_b']
    for i, col in enumerate(waste_cols):
        fig.add_trace(go.Bar(x=df['period'], y=df[col], name=col, marker_color=px.colors.qualitative.Pastel[i]), row=2, col=1)
    total_alloc = df[['X_f', 'X_a', 'X_b']].sum()
    fig.add_trace(go.Pie(labels=['生鮮(f)', 'タルト用(a)', 'シロップ用(b)'], values=total_alloc, hole=.3), row=2, col=2)
    fig.update_layout(height=800, title_text=f"詳細シミュレーションダッシュボード (λ = {df['weight'].iloc[0]})", legend_title_text="凡例")
    fig.update_xaxes(title_text="期間")
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=1)
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=2)
    fig.update_yaxes(title_text="重量 (kg)", row=2, col=1)
    fig.show()

# =================================================================
# モジュール8: メイン実行ブロック
# =================================================================
if __name__ == '__main__':
    SIMULATION_PERIODS = 12

    # パレート分析で使用する「共通の」需要シナリオセットを事前に生成する
    print("="*80)
    print(f"{SIMULATION_PERIODS}期間分の共通需要シナリオ（マスターシナリオ）を生成しています...")
    print("="*80)
    master_scenarios = []
    for t in range(SIMULATION_PERIODS):
        scenarios_for_period_t = {
            f"s_{i+1}": {
                'prob': 1 / num_scenarios_per_period,
                'd': {p: max(0, np.random.normal(demand_params['mean'][p], demand_params['std'][p])) for p in products}
            } for i in range(num_scenarios_per_period)
        }
        master_scenarios.append(scenarios_for_period_t)
    print("マスターシナリオの生成が完了しました。\n")

    # Numpyライブラリによる対数スケール(10の何乗か)指定(開始点, 終了点, 生成数)
    # 環境負荷１ポイントにつき0.1円～1000円
    # --- 対数スケールで広範囲をスキャン ---
    # env_weights_log = np.logspace(-1, 3, 20)
    # df_results_log = run_pareto_analysis(env_weights_log, simulation_periods=SIMULATION_PERIODS, title=" 広域スキャン（対数スケール）")

    # if not df_results_log.empty:
    #   # --- グラフ1: 対数スケールでの結果 ---
    #   fig_lambda_log = make_subplots(specs=[[{"secondary_y": True}]])
    #   fig_lambda_log.add_trace(go.Scatter(x=df_results_log['weight'], y=df_results_log['total_cost'], name='総コスト'), secondary_y=False)
    #   fig_lambda_log.add_trace(go.Scatter(x=df_results_log['weight'], y=df_results_log['total_env_score'], name='総環境負荷'), secondary_y=True)
    #   fig_lambda_log.update_layout(title_text='【ステージ1】広域スキャン: λと各指標の関係', xaxis_type="log")
    #   fig_lambda_log.update_xaxes(title_text='λ (環境重視度) ※対数スケール')
    #   fig_lambda_log.update_yaxes(title_text="総コスト (円)", secondary_y=False)
    #   fig_lambda_log.update_yaxes(title_text="総環境負荷スコア", secondary_y=True)
    #   print("\n--- グラフ1: 広域スキャンの結果 ---")
    #   fig_lambda_log.show()

    # --- 等間隔スケールで特定範囲を詳細スキャン ---
    # 研究発表で議論したい、変化の大きい範囲を線形スケールで分析
    zoom_range_start = 0
    zoom_range_end = 400
    num_points_zoom = 20

    env_weights_linear = np.linspace(zoom_range_start, zoom_range_end, num_points_zoom)
    df_results_linear = run_pareto_analysis(env_weights_linear, simulation_periods=SIMULATION_PERIODS, master_scenarios=master_scenarios, title=f"詳細スキャン（λ={zoom_range_start}～{zoom_range_end}）")

    if not df_results_linear.empty:
        fig_lambda_linear = make_subplots(specs=[[{"secondary_y": True}]])
        fig_lambda_linear.add_trace(go.Scatter(x=df_results_linear['weight'], y=df_results_linear['total_cost'], name='総コスト', mode='lines+markers'), secondary_y=False)
        fig_lambda_linear.add_trace(go.Scatter(x=df_results_linear['weight'], y=df_results_linear['total_env_score'], name='総環境負荷', mode='lines+markers'), secondary_y=True)
        fig_lambda_linear.update_layout(title_text=f'【ステージ2】詳細スキャン: λ={zoom_range_start}～{zoom_range_end}における各指標の関係')
        fig_lambda_linear.update_xaxes(title_text='λ (環境重視度) ※等間隔スケール')
        fig_lambda_linear.update_yaxes(title_text="総コスト (円)", secondary_y=False)
        fig_lambda_linear.update_yaxes(title_text="総環境負荷スコア", secondary_y=True)
        print("\n--- グラフ2: 詳細スキャンの結果 ---")
        fig_lambda_linear.show()

        print("\n" + "="*80)
        print("【詳細スキャン範囲の比較表】")
        print("="*80)
        display_columns = ['weight', 'total_cost', 'total_env_score', 'total_objective']
        print(df_results_linear[display_columns].round(2))
        print("="*80)

    # --- ステップ3: 選択したλで詳細分析 (例: λ=100) ---
    chosen_weight = 100.0 # ステージ2の結果を見て、研究者が決定する
    print("\n" + "="*80)
    print(f"ステップ3: 選択したλ={chosen_weight} での詳細分析")
    print("="*80)

    # 1期間目の詳細分析
    print("--- 1期間目の詳細分析 ---")
    first_period_scenarios = master_scenarios[0] # ★共通シナリオの1期目を渡す
    _, first_period_model = solve_multi_objective_model(initial_inventory_post, initial_inventory_pre, C_supply, first_period_scenarios, params, capacities, chosen_weight)
    if first_period_model:
        analyze_and_print_period_details(first_period_model, first_period_scenarios, params)
    else:
        print("詳細分析のための最適解が見つかりませんでした。")

    # 12ヶ月間のダッシュボード表示
    print("\n--- 12ヶ月間のダッシュボード ---")
    detailed_df = run_rolling_horizon_simulation(simulation_periods=SIMULATION_PERIODS, env_weight=chosen_weight,
                                               initial_inventory_post=initial_inventory_post,
                                               initial_inventory_pre=initial_inventory_pre,
                                               master_scenarios=master_scenarios, # ★共通シナリオを渡す
                                               verbose=False)

    if not detailed_df.empty:
        print(f"総コスト: {detailed_df['cost'].sum():,.2f} 円")
        print(f"総環境負荷スコア: {detailed_df['env_score'].sum():,.2f}")
        print("\nダッシュボードを生成しています...")
        create_dashboard(detailed_df)


12期間分の共通需要シナリオ（マスターシナリオ）を生成しています...
マスターシナリオの生成が完了しました。

詳細スキャン（λ=0～400）を開始...

完了: 17.32 秒

--- グラフ2: 詳細スキャンの結果 ---



【詳細スキャン範囲の比較表】
    weight  total_cost  total_env_score  total_objective
0     0.00   166301.25          1053.12        166301.25
1    21.05   169078.28          1038.46        190940.53
2    42.11   172437.32          1030.06        215808.25
3    63.16   180841.13          1017.11        245079.60
4    84.21   193042.46          1004.07        277595.55
5   105.26   208630.31           999.53        313843.82
6   126.32   229705.09           954.19        350234.46
7   147.37   250503.97           946.24        389949.63
8   168.42   276949.16           934.43        434327.11
9   189.47   322945.46           924.59        498130.29
10  210.53   367538.72           921.15        561464.31
11  231.58   403328.40           923.31        617147.25
12  252.63   431908.72           899.15        659061.88
13  273.68   462145.75           883.34        703902.93
14  294.74   505812.76           862.60        760051.62
15  315.79   588288.92           814.37        845459.96
16  336.84   58

- 環境負荷を制約とする（環境負荷＜＝上限以下 とする）
- 重みづけて一つの目的関数とする（重みを付けて一に）
  コスとを1とした場合の重みづけになっているので、コスト+環境負荷を足して同時に計算する
- 対話的に求める（アルゴリズム）（満足しない場合→どちらかを減らす→満足しない→どちらかを減らす...を満足するまで繰り返す）多目的対話、対話的に求めるかどうかは難しい